## Project: Applying NLP to Major Tasks
#### PROG8245- Machine Learning Programming

| Professor     | Islam Mahmoud               |
|---------------|-----------------------------|
| Email         | imahmoud@conestogac.on.ca   |
| Deadline      | Monday, December 9th, 2024  |
| Group         | Group 10                    |
| Members       | Burman, Jaiv Chaitanya      |
|               | Lad, Khushbu Nileshkumar    |
|               | Madhav, Akshata Ajay        |


### Data Collection

#### Collect a dataset of product reviews. 

#### Annotate the dataset with labels of positive, negative or neutral sentiment, based on collected data.

### Preprocessing

#### Perform necessary text preprocessing steps such as tokenization, stop-word removal, stemming/lemmatization, and lowercasing. 

#### Remove any irrelevant columns, handle missing values, and clean text data by removing special characters, stopwords, and performing stemming/lemmatization

#### Handle specific challenges of used text like hashtags, emojis, and slang.

### Feature Extraction and Model Comparison

#### Explore different feature representation methods such as bag-of-words, TF-IDF, word embeddings (e.g., Word2Vec or GloVe), or contextual embeddings (e.g., BERT or GPT). Experiment with 3 different feature extraction techniques to capture meaningful representations of social media text where the 3 techniques should be of different word embedding categories.

#### Model Building
- Choose a suitable machine learning algorithm (e.g., Naive Bayes, SVM, or neural networks) or deep learning model
- Split the dataset into training and testing sets.
- Train the selected model using the training data, evaluate and record its performance on the training and testing data.

#### Interpretation of results
- Visualize your results
- Compare different feature representation
- Reach a conclusion on which is the best Embedding technique to use for your model